In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
filepath = "/home/veroastro/Documents/NASA_POWER_ds/data/"
df = pd.read_csv(filepath + "POWER_Point_Hourly_20010101_20211231_041d39N_002d17E_UTC.csv")
df.head()

,YEAR,MO,DY,HR,WD10M,T2MWET,T2MDEW,PS,WS10M,V10M,U10M,QV2M,T2M,RH2M,PRECTOTCORR
0,2001,1,1,0,286.64,1.42,-0.66,97.95,3.00,-0.86,2.88,3.85,3.49,76.50,0.0
1,2001,1,1,1,295.52,1.70,-0.24,97.89,2.87,-1.23,2.59,3.91,3.65,77.62,0.0
2,2001,1,1,2,296.91,1.70,-0.01,97.82,2.90,-1.31,2.59,3.97,3.41,80.12,0.0
3,2001,1,1,3,293.30,2.04,0.51,97.76,2.88,-1.14,2.65,4.15,3.55,82.19,0.0
4,2001,1,1,4,295.29,2.64,1.36,97.71,2.82,-1.20,2.55,4.39,3.92,84.69,0.0


In [6]:
df_new = df.drop(['T2MWET','T2MDEW', 'PS'], axis = 1) # dropped to deal with multicollinearity

In [7]:
from sklearn.preprocessing import MinMaxScaler
X = df_new.drop('T2M', axis= 1)
y = df_new['T2M']
scaler = MinMaxScaler()
X_t = scaler.fit_transform(X)
X.columns

Index(['YEAR', 'MO', 'DY', 'HR', 'WD10M', 'WS10M', 'V10M', 'U10M', 'QV2M',
       'RH2M', 'PRECTOTCORR'],
      dtype='object')

In [8]:
# Splitting the Xand y datasets into training and test set and normalising the data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_t, y, 
                                                    test_size=0.3, random_state=42)

In [ ]:
# Random Forest with GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
RF = RandomForestRegressor(random_state = 42)
param_grid = {  'bootstrap': [True, False],
                'max_features': ['auto', 'sqrt','log2'], 
                'n_estimators': range(100,400, 20)}

rf= GridSearchCV(RF,
                  param_grid=param_grid)

# Train the model on training data
rf.fit(X_train, y_train);



In [ ]:
predictions = rf.predict(X_test)

In [ ]:
rf.best_params_ ,rf.best_score_